In [ ]:
from pylab import *
import os, sys, tarfile, base64
os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from random import randint
from math import sqrt
#from keras import backend as K
from tensorflow.python.ops import gen_nn_ops

import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.applications import *
from tensorflow.keras.losses import *
from tensorflow.keras.metrics import * 
from tensorflow.keras.regularizers import *
from tensorflow.keras.activations import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K 
from tensorflow_addons.optimizers import Lookahead, RectifiedAdam
from tensorflow.keras.preprocessing.image import *
import tensorflow_addons as tfa
from tensorflow.keras.utils import *
from skimage.io import imread
from skimage.transform import resize
import cv2

In [ ]:
newcollege_images_path = "./NewCollege"
newcollege = sorted(
    [str(newcollege_images_path +"/"+ f) for f in os.listdir(newcollege_images_path)]
)
citycentre_images_path = "./CityCentre"

citycentre = sorted(
    [str(citycentre_images_path +"/"+ f) for f in os.listdir(citycentre_images_path)]
)

In [ ]:
newcollege_gt = np.loadtxt("./drive/MyDrive/NewCollegeGroundTruth.txt", delimiter=',')
citycentre_gt = np.loadtxt("./drive/MyDrive/CityCentreGroundTruth.txt", delimiter=',')

In [ ]:
with open("./drive/MyDrive/NewCollegeGPS.txt", 'r') as f:
    newcollege_gps = [line.split()[1:] for line in f]   
# Converting to float for further work
newcollege_gps = np.array(newcollege_gps, dtype=np.float32)
# Deleting all the NaN values from the set - this problem only occurs in New College
newcollege_gps = newcollege_gps[:1984]
print(f"New size New College: {len(newcollege_gps)}")
with open("./drive/MyDrive/CityCentreGPS.txt", 'r') as f:
    citycentre_gps = [line.split()[1:] for line in f] 
# Converting to float for further work
citycentre_gps = np.array(citycentre_gps, dtype=np.float32)

In [ ]:
# Focal length:
fc = [ 367.481519978327754 , 366.991059667167065 ]
# Principal point:
cc = [ 328.535778962615268 , 233.779960757465176 ]
# Distortion coefficients:
kc = [ -0.293510277812333 ,
        0.065334967950619 ,
       -0.000117308680498 ,
        0.000304779905426 ,
        0.000000000000000 ]

im_width  = 640
im_height = 480

In [ ]:
import random

def add_random_mask(image, mask_size=50, mask_count=10):

    height, width, _ = image.shape

    mask = np.zeros((height, width), dtype=np.uint8)

    for _ in range(mask_count):
        x = random.randint(0, width - mask_size)
        y = random.randint(0, height - mask_size)
 
        mask[y:y + mask_size, x:x + mask_size] = 255
    
    masked_image = image.copy()
    masked_image[mask == 255] = [0, 0, 0]  

    return masked_image

In [ ]:
def distanceP(a, b):
    return sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2)

In [ ]:
def createInputOutput(imageset, gt, gps):

    
    l = len(imageset)    
    data_n1 = []
    data_n2 = []
    data_p1 = []
    data_p2 = []
    
    DELTA_MAX = 3
    
    DELTA_MIN = 200
   
    l = min(l, len(gps))
    for st in range(0,2):
        for i in range(st,l,2):
            for j in range(st,l,2):
                if float(gt[i][j]) > 0 and distanceP(gps[i], gps[j]) <= DELTA_MAX:
                    data_p1.append(i)
                    data_p2.append(j)
                    
                elif distanceP(gps[i], gps[j]) >= DELTA_MIN:
                    data_n1.append(i)
                    data_n2.append(j)
                            
    ridx = np.random.permutation(len(data_n1))
    data_n1 = int32(data_n1)[ridx].copy()
    data_n2 = int32(data_n2)[ridx].copy()
    
    print(f'After normalising: Closures {len(data_p1)}, Non-closures {len(data_n1)}') 
    data1n, data2n, outputn = unison_shuffle_triple(data_n1, data_n2, zeros_like(data_n1))
    data1p, data2p, outputp = unison_shuffle_triple(data_p1, data_p2, ones_like(data_p1))
    return (data1n, data1p), (data2n, data2p), (outputn, outputp)
    

In [ ]:
def createInputOutput(imageset, gt, gps, mask_size=50, mask_count=5):
    l = len(imageset)    
    data_n1 = []
    data_n2 = []
    data_p1 = []
    data_p2 = []
    
    DELTA_MAX = 3
    DELTA_MIN = 200
    
    l = min(l, len(gps))
    for st in range(0, 2):
        for i in range(st, l, 2):
            for j in range(st, l, 2):
                if float(gt[i][j]) > 0 and distanceP(gps[i], gps[j]) <= DELTA_MAX:
                    data_p1.append(i)
                    data_p2.append(j)
                    
                elif distanceP(gps[i], gps[j]) >= DELTA_MIN:
                    data_n1.append(i)
                    data_n2.append(j)
    
    ridx = np.random.permutation(len(data_n1))
    data_n1 = np.int32(data_n1)[ridx].copy()
    data_n2 = np.int32(data_n2)[ridx].copy()
    
    print(f'After normalising: Closures {len(data_p1)}, Non-closures {len(data_n1)}')
     
    data1n, data2n, outputn = unison_shuffle_triple(data_n1, data_n2, np.zeros_like(data_n1))
    data1p, data2p, outputp = unison_shuffle_triple(data_p1, data_p2, np.ones_like(data_p1))
    
    # Now add random masks to the image data
    imageset_n1 = [add_random_mask(imageset[i], mask_size, mask_count) for i in data_n1]
    imageset_n2 = [add_random_mask(imageset[i], mask_size, mask_count) for i in data_n2]
    imageset_p1 = [add_random_mask(imageset[i], mask_size, mask_count) for i in data_p1]
    imageset_p2 = [add_random_mask(imageset[i], mask_size, mask_count) for i in data_p2]
    
    return (imageset_n1, imageset_n2), (imageset_p1, imageset_p2), (outputn, outputp)

In [ ]:
inputData1NC, inputData2NC, outputDataNC = createInputOutput(newcollege, newcollege_gt, newcollege_gps)
inputData1CC, inputData2CC, outputDataCC = createInputOutput(citycentre, citycentre_gt, citycentre_gps)
trainData1 = inputData1CC
trainData2 = inputData2CC
trainData3 = inputData1CC
trainDataOut = outputDataCC
valData1 = inputData1NC
valData2 = inputData2NC
valData3 = inputData1NC
valDataOut = outputDataNC

In [ ]:
# class CapsLayer(Layer):
#     def __init__(self, privateCaps_dim,outputCaps_num,outputCaps_dim,r=3):
#         super(CapsLayer, self).__init__()
#         self.outputCaps_num = outputCaps_num
#         self.outputCaps_dim = outputCaps_dim
#         self.privateCaps_dim = privateCaps_dim
#         self.r = r
#         self.epsilon = 1e-9

#     def build(self, input_shape):
#         self.w_ij = self.add_weight(shape=(1,self.outputCaps_num,int(input_shape[2]),int(input_shape[3]),self.outputCaps_dim),
#                                      initializer=tf.keras.initializers.RandomNormal(),
#                                      trainable=True,name='w_ij')
    
#     def call(self, u_i):
#         u_i = tf.reshape(u_i,shape=(-1,1,u_i.shape[1]*u_i.shape[2]*u_i.shape[3]//self.privateCaps_dim,self.privateCaps_dim,1))
#         b_ij = tf.zeros([self.outputCaps_num,u_i.shape[2]//1,1,1],name='b_ij')
#         for _ in range(self.r):
#             u_hat = tf.matmul(self.w_ij,u_i,transpose_a=True,name='u_hat')
#             #print(u_hat.shape)
#             c_ij = tf.nn.softmax(b_ij,axis=1,name='c_ij')
#             #print(c_ij.shape)
#             s_j = tf.reduce_sum(tf.multiply(c_ij,u_hat),axis=2,keepdims=True,name='s_j')
#             #print(s_j.shape)
#             v_j = self.squash(s_j)
#             #print(v_j.shape)
#             update = tf.reduce_mean(tf.matmul(u_hat,v_j,transpose_a=True),axis=0)
#             #print(update)
#             b_ij += update
#             #print(b_ij)
        
#         return v_j
    
#     def squash(self,vector):
#         '''Squashing function corresponding to Eq. 1
#         Args:
#             vector: A tensor with shape [batch_size, 1, num_caps, vec_len, 1] or [batch_size, num_caps, vec_len, 1].
#         Returns:
#             A tensor with the same shape as vector but squashed in 'vec_len' dimension.
#         '''
#         vec_squared_norm = tf.reduce_sum(tf.square(vector), -2, keepdims=True)
#         scalar_factor = vec_squared_norm / (1 + vec_squared_norm) / tf.sqrt(vec_squared_norm + self.epsilon)
#         vec_squashed = scalar_factor * vector  # element-wise
#         return(vec_squashed)

In [ ]:
def squash(x, axis=-1):
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    scale = K.sqrt(s_squared_norm) / (0.5 + s_squared_norm)
    return scale * x
def softmax(x, axis=-1):
    ex = K.exp(x - K.max(x, axis=axis, keepdims=True))
    return ex / K.sum(ex, axis=axis, keepdims=True)
def caps_batch_dot(x, y):
    x = K.expand_dims(x, 2)
    if K.int_shape(x)[3] is not None:
        y = K.permute_dimensions(y, (0, 1, 3, 2))
    o = tf.matmul(x, y)
    return K.squeeze(o, 2)

class Capsule(Layer):
    def __init__(self,
                 num_capsule,
                 dim_capsule,
                 routings=3,
                 share_weights=True,
                 activation='squash',
                 **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.share_weights = share_weights
        if activation == 'squash':
            self.activation = squash
        else:
            self.activation = activations.get(activation)

    def build(self, input_shape):
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(1, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)
        else:
            if input_shape[-2] is None:
                raise ValueError("Input Shape must be defied if weights not shared.")
            input_num_capsule = input_shape[-2]
            self.kernel = self.add_weight(
                name='capsule_kernel',
                shape=(input_num_capsule, input_dim_capsule,
                       self.num_capsule * self.dim_capsule),
                initializer='glorot_uniform',
                trainable=True)

    def call(self, inputs):

        if self.share_weights:
            hat_inputs = K.conv1d(inputs, self.kernel)
        else:
            hat_inputs = K.local_conv1d(inputs, self.kernel, [1], [1])

        batch_size = K.shape(inputs)[0]
        input_num_capsule = K.shape(inputs)[1]
        hat_inputs = K.reshape(hat_inputs,
                               (batch_size, input_num_capsule,
                                self.num_capsule, self.dim_capsule))
        hat_inputs = K.permute_dimensions(hat_inputs, (0, 2, 1, 3))

        b = K.zeros_like(hat_inputs[:, :, :, 0])

        
        for i in range(self.routings):
            c = softmax(b, 1)
            o = self.activation(caps_batch_dot(c, hat_inputs))
            if i < self.routings - 1:
                b = caps_batch_dot(o, hat_inputs)
                if K.backend() == 'theano':
                    o = K.sum(o, axis=1)
        return o

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)
        
    def get_config(self):
        config = {
            'num_capsule': self.num_capsule,
            'dim_capsule': self.dim_capsule,
            'routings': self.routings
        }
        base_config = super(Capsule, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

In [ ]:
import keras
from keras import backend as k
from keras import initializers
from keras import regularizers

va = Input((224,224,3))
vb = Input((224,224,3))

from keras import layers, Input, Model, optimizers
import keras.backend as K
def ACT():
    """
    Activation function.
    """
    return Activation('relu')
# A Siamese branch part 
vggmodel = Sequential([
         Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
         Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"),
         MaxPooling2D(pool_size=(2,2),strides=2,name="MP1"),
         Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"),
         Conv2D(filters=128,kernel_size=(3,3),padding="same", activation="relu"),
         MaxPooling2D(pool_size=(2,2),strides=2,name="MP2"),
         Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"),
         Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"),
         Conv2D(filters=256,kernel_size=(3,3),padding="same", activation="relu"),
         MaxPooling2D(pool_size=(2,2),strides=2,name="MP3"),
         Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"),
         Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"),
         Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"),
         MaxPooling2D(pool_size=(2,2),strides=2,name="MP4"),
         Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"),
         Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"),
         Conv2D(filters=512,kernel_size=(3,3),padding="same", activation="relu"),
         MaxPooling2D(pool_size=(2,2),strides=2,name="MP5"),
         Dense(4096),
         Dense(4096),
         Dense(1000),
         BatchNormalization(),
         Flatten(),
      
])
encoded_image_1 = vggmodel(va) 
encoded_image_2 = vggmodel(vb)
descriptor = concatenate([encoded_image_1, encoded_image_2])
final_layer = Flatten()(descriptor)
final_layer = Dense(512,activation=ACT())(final_layer)
final_layer = BatchNormalization()(final_layer)
final_layer = Dense(128,activation=ACT())(final_layer) 
final_layer = BatchNormalization()(final_layer)
final_layer = Dense(64,activation=ACT())(final_layer)
final_layer = BatchNormalization()(final_layer)
final_layer = Dense(1, activation='sigmoid')(final_layer)
siamese_vgg = Model(inputs=[va,vb], outputs = final_layer)
siamese_vgg.compile(
    loss='binary_crossentropy', 
    optimizer=Lookahead(RectifiedAdam(learning_rate = 0.01)),
    metrics=["accuracy", 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall(), 
             tf.keras.metrics.AUC(curve="PR")]
)

siamese_vgg.summary()

In [ ]:
aa = Input((96,96,3))
ab = Input((96,96,3))

from keras import layers, Input, Model, optimizers
import keras.backend as K
def ACT():
    """
    Activation function.
    """
    return Activation('relu')
# A Siamese branch part 
alexmodel = Sequential([
         Conv2D(filters=96,  kernel_size=11, strides=4, padding="same",name="conv1"),ACT(),
         MaxPooling2D(pool_size=(3,3),strides=2,name="MP1"),
         Conv2D(filters=256, kernel_size=(5,5),padding='same',strides=1,name="conv2"), ACT(),
         MaxPooling2D(pool_size = (3,3),strides=2,name="MP2"),
         Conv2D(filters=384, kernel_size=(3,3),padding='same',strides=1,name="conv3"),ACT(),
         Conv2D(filters=384, kernel_size=(3,3),padding='same',strides=1,name="conv4"),ACT(),
         Conv2D(filters=256, kernel_size=(3,3),padding='same',strides=1,name="conv5"),ACT(),
         MaxPooling2D(pool_size = (3,3),strides=2,name="MP3"),
         Dense(4096),
         Dense(4096),
         Dense(100),
         Flatten(),
      
])
encoded_image_1 = alexmodel(aa) 
encoded_image_2 = alexmodel(ab)
descriptor = concatenate([encoded_image_1, encoded_image_2])
final_layer = Flatten()(descriptor)
final_layer = Dense(128,activation=ACT())(final_layer)
final_layer = BatchNormalization()(final_layer)
final_layer = Dense(1, activation='sigmoid')(final_layer)
siamese_alex = Model(inputs=[aa,ab], outputs = final_layer)
siamese_alex.compile(
    loss='binary_crossentropy', 
    optimizer=Lookahead(RectifiedAdam(learning_rate = 0.004)),
    metrics=["accuracy", 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall(), 
             tf.keras.metrics.AUC(curve="PR")]
)

siamese_alex.summary()

In [ ]:

c = Input((96,128,3))
a = Input((224,224,3))
b = Input((224,224,3))
def ACT():
    """
    Activation function.
    """
    return Activation('relu')
    
def triplet_loss(x, alpha = 0.2):
    # Triplet Loss function.
    anchor,positive,negative = x
    # distance between the anchor and the positive
    pos_dist = K.sum(K.square(anchor-positive),axis=1)
    # distance between the anchor and the negative
    neg_dist = K.sum(K.square(anchor-negative),axis=1)
    # compute loss
    basic_loss = pos_dist-neg_dist+alpha
    loss = K.maximum(basic_loss,0.0)
    return loss
# A Siamese branch part 
model = Sequential([
      Conv2D(filters=96,  kernel_size=11, strides=4), BatchNormalization(), ACT(),
      MaxPool2D(pool_size=(3,3),strides=2),
      Conv2D(filters=256, kernel_size=(5,5),padding='same',strides=1), ACT(),
      MaxPool2D(pool_size = (3,3),strides=2),
      Conv2D(filters=384, kernel_size=(3,3),padding='same',strides=1),ACT(),
      Conv2D(filters=384, kernel_size=(3,3),padding='same',strides=1),ACT(),
      Conv2D(filters=256, kernel_size=(3,3),padding='valid',strides=1),ACT(),
      MaxPool2D(pool_size = (3,3),strides=2),
      Flatten(),
])
# model_vgg = Sequential([
#     Conv2D(64, (3, 3), padding='same', activation='relu'),
#     Conv2D(64, (3, 3), activation='relu', padding='same'),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(128, (3, 3), activation='relu', padding='same'),
#     Conv2D(128, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     Conv2D(256, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
#     Flatten(),
#     # Dense(4096, activation='relu'),
#     # Dense(4096, activation='relu'),
#     # Dense(1000, activation='softmax')
# ])
# ————————————————
encoded_image_1 = model(a) 
encoded_image_2 = model(b)
#encoded_image_3 = model(c)
# vgg_image_1 = model_vgg(va) 
# vgg_image_2 = model_vgg(vb)

# Concatenate the resulting low-dimensional descriptors into one
#descriptor = ([encoded_image_1, encoded_image_2,encoded_image_3])
descriptor = concatenate([encoded_image_1, encoded_image_2])
#descriptor_vgg = ==([vgg_image_1, vgg_image_2])
# The dense "decision" layers
final_layer = Flatten()(descriptor)
final_layer = Dense(4096,activation=ACT())(final_layer)
final_layer = BatchNormalization()(final_layer)
final_layer = Dropout(0.5)(final_layer)
final_layer = Dense(4096,activation=ACT())(final_layer) 
final_layer = BatchNormalization()(final_layer)
final_layer = Dropout(0.5)(final_layer)
final_layer = Dense(1024,activation=ACT())(final_layer) 
final_layer = BatchNormalization()(final_layer)
final_layer = Dense(128,activation=ACT())(final_layer)
final_layer = BatchNormalization()(final_layer)
final_layer = Dense(1, activation='softmax')(final_layer)


#---------vgg 
# final_layer_vgg=Flatten()(descriptor_vgg)
# final_layer_vgg=Dense(4096,activation=ACT())(final_layer_vgg)
# final_layer_vgg=BatchNormalization()(final_layer_vgg)
# final_layer_vgg=Dense(4096,activation=ACT())(final_layer_vgg)
# final_layer_vgg=BatchNormalization()(final_layer_vgg)
# final_layer_vgg=Dense(1000,activation='sigmoid')(final_layer_vgg)
# final_layer_vgg=BatchNormalization()(final_layer_vgg)
# final_layer_vgg = Dense(1, activation='softmax')(final_layer)
# Assemble the entire neural network together
#siamese_autoencoder = Model(inputs=[a,b,c], outputs = final_layer)
siamese_autoencoder = Model(inputs=[a,b], outputs = final_layer)
#siamese_autoencoder_vgg=Model(input=[va,vb],outputs = final_layer_vgg)
# Make the Siamese branches trainable (change both True to False otherwise)
for i in range(len(model.layers)-1):
    model.layers[i].trainable = True
model.trainable = True

# for i in range(len(model_vgg.layers)-1):
#     model_vgg.layers[i].trainable = True
# model_vgg.trainable = True
# Compile the final model
siamese_autoencoder.compile(
    loss='binary_crossentropy', 
    #loss=Lambda(triplet_loss)([encoded_image_1, encoded_image_2,encoded_image_3]) 
    optimizer=Lookahead(RectifiedAdam(lr = 0.01)),
    metrics=["accuracy", 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall(), 
             tf.keras.metrics.AUC(curve="PR")]
)

# Print the summary of the main model and the Siamese branches
siamese_autoencoder.summary()
model.summary()
# siamese_autoencoder_vgg.summary()
# model_vgg.summary()
  
# Set the coefficients for the branches to the pre-trained ones
# for i in range(len(model.layers)-1):
#     model.layers[i].set_weights(aemodel.layers[i].get_weights())


# import psutil
# import humanize
# import os
# #!pip install gputil
# import GPUtil as GPU

# GPUs = GPU.getGPUs()
# # XXX: only one GPU on Colab and isn’t guaranteed
# gpu = GPUs[0]
# def printm():
#     process = psutil.Process(os.getpid())
#     print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
#     print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
# printm()
# !ps -aux|grep python
# !kill -9 70
# !nvidia-smi

In [ ]:
input_image = Input(shape=(96, 96, 3))
x = Conv2D(64, (3, 3))(input_image)
x=layers.Activation('relu')(x)
x=BatchNormalization()(x)
x = Conv2D(64, (3, 3))(x)
x=layers.Activation('relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(128, (3, 3))(x)
x=layers.Activation('relu')(x)
x = Conv2D(128, (3, 3))(x)
x=layers.Activation('relu')(x)
x = Reshape((-1, 128))(x)
x = Capsule(32, 8, 3, True)(x)  
#x = Capsule(32, 8, 3, True)(x)   
capsule = Capsule(8, 16, 3, True)(x)
output = Lambda(lambda z: K.sqrt(K.sum(K.square(z), 2)))(capsule)
# output- Dense(96)(output)
capsnet = Model(inputs=[input_image], outputs=[output])
#..............................................................................................

#.............................................................................................
# capsnet=CapsNet()
caps1=capsnet(c)
caps2=capsnet(b)
#descriptor = concatenate([caps1,caps2])
L1_layer = Lambda(lambda tensors: K.abs(tensors[0] - tensors[1]))
L1_distance = L1_layer([caps1, caps2])
final_layer = BatchNormalization()(L1_distance)
final_layer = Dense(1, activation='sigmoid')(final_layer)
emb_dimentions=96
# Assemble the entire neural network together
siamese_cap = Model(inputs=[c,b], outputs = final_layer)

# Make the Siamese branches trainable (change both True to False otherwise)
for i in range(len(capsnet.layers)-1):
    capsnet.layers[i].trainable = True
capsnet.trainable = True

# Compile the final model
siamese_cap.compile(
    loss='binary_crossentropy', 
    #loss = triplet_loss(alpha=0.4,emb_dimentions=emb_dimentions),
#     optimizer=Adam(learning_rate = 0.00006),
   optimizer=Lookahead(RectifiedAdam(learning_rate = 0.004)),
    metrics=["accuracy", 
             tf.keras.metrics.Precision(), 
             tf.keras.metrics.Recall(), 
             tf.keras.metrics.AUC(curve="PR")]
#     metrics=METRICS
)

In [ ]:
class DataSequence(tf.keras.utils.Sequence):
    def __init__(self, imageset, data1, data2,output, batch_size=256, aug=True):
        super().__init__()
        self.data1 = data1
        self.data2 = data2
        #self.data3 = data3
        self.output = output
        self.batch_size = batch_size
        self.internalCounter = 0
        self.imageset = imageset
        self.aug = aug

    def __len__(self):
        return math.ceil(len(self.data1) / self.batch_size)
    
    def __getitem__(self, idx):
        start = idx * self.batch_size
        insA = [ ]
        insB = [ ]
        #insC = [ ]
        outs = [ ]
        maxel = self.batch_size
        for i in range(maxel):
            q = randint(0,1)
            ridx = randint(0, len(self.data1[q])-1)
            if self.aug and q==1 and rand()<0.1: # every Nth image is the same
                q = 0
                ridx = randint(0, len(self.data1[q])-1)
                insA.append(augment(preprocess_image(self.imageset[self.data1[q][ridx]])))
                insB.append(augment(preprocess_image(self.imageset[self.data1[q][ridx]])))
                #insC.append(augment(preprocess_image(self.imageset[self.data1[q][ridx]])))
                outs.append(1.0)
            else:                
                if self.aug:
                    insA.append(augment(preprocess_image(self.imageset[self.data1[q][ridx]])))
                    insB.append(augment(preprocess_image(self.imageset[self.data2[q][ridx]])))
                    #insC.append(augment(preprocess_image(self.imageset[self.data3[q][ridx]])))
                else:
                    insA.append(preprocess_image(self.imageset[self.data1[q][ridx]]))
                    insB.append(preprocess_image(self.imageset[self.data2[q][ridx]]))
                    #insC.append(preprocess_image(self.imageset[self.data3[q][ridx]]))
                outs.append(self.output[q][ridx])
        
        return (float32(insA),float32(insB)), float32(outs)

x_train = DataSequence(citycentre, trainData1, trainData2, trainDataOut, batch_size=200)    
x_val   = DataSequence(newcollege, valData1, valData2, valDataOut,   batch_size=100, aug=False)   


# Generator as a function
def funcGen(gen):
    while(True):
        for ex in gen:
            yield ex

In [ ]:
for step in range(100):
    history = siamese_autoencoder.fit(
        funcGen(x_train),
        epochs=100,
        #batch_size=16,
        steps_per_epoch=100,
        validation_data = funcGen(x_val),
        validation_steps = 100,
        callbacks = [
            ReduceLROnPlateau(
                monitor="val_loss",
                factor=0.2,
                patience=5,
                verbose=1,
            ),
            EarlyStopping(
                monitor="val_loss",
                patience=5,
                verbose=1,
                restore_best_weights=True,
            ),
            ModelCheckpoint(
                'alex-Loss-cc-nc-g-checkpoint',
                monitor="val_loss",
                save_best_only=True,
                save_weights_only=True,
            ),
        ]
    )